In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('coord with state.csv',header = 0, na_values = '?',encoding='latin-1') #read data from csv
labels = ['year','HD2016.FIPS state code','HD2016.State abbreviation','State .1','Unnamed: 10','Unnamed: 11','Code']
df = df.drop(labels = labels, axis = 1) #dropped columns that are not gonna be used
columns ={"unitid": "id", "institution name": "name","HD2016.Longitude location of institution":"longitude","HD2016.Latitude location of institution":"latitude", "State ":"state"}
dfCollege = df.rename(index=str, columns=columns) #renamed columns
dfCollege.head(6)



,id,name,longitude,latitude,state
0,100654,Alabama A & M University,-86.568502,34.783368,AL
1,100663,University of Alabama at Birmingham,-86.799345,33.505697,AL
2,100690,Amridge University,-86.174010,32.362609,AL
3,100706,University of Alabama in Huntsville,-86.640449,34.724557,AL
4,100724,Alabama State University,-86.295677,32.364317,AL
5,100733,University of Alabama System Office,-87.529594,33.207015,AL


In [30]:
#added additional columns for calculating distance from college campuses
dfRest = pd.read_csv('rest_data_state.csv', header=0, sep=',', index_col=0, encoding='latin-1')
dfRest['min_dist_1'] = ""
dfRest['min_dist_2'] = ""
dfRest['min_dist_3'] = ""
dfRest['min_dist_4'] = ""
dfRest['min_dist_5'] = ""
dfRest.head(6)

,business_id,latitude,longitude,review_count,stars,state,min_dist_1,min_dist_2,min_dist_3,min_dist_4,min_dist_5
0,YDf95gJZaq05wvo7hTQbbQ,41.541716,-81.493116,17,2.0,OH,,,,,
1,mLwM-h2YhXl2NCgdS84_Bw,35.236870,-80.741976,4,4.5,NC,,,,,
2,v2WhjAB3PIBA8J8VxG3wEg,43.677126,-79.353285,7,4.5,ON,,,,,
3,CVtCbSB1zUcUWg-9TNGTuQ,33.565082,-111.916400,3,3.0,AZ,,,,,
4,duHFBe87uNSXImQmvBh87Q,33.505928,-112.038847,10,4.5,AZ,,,,,
5,uUEMrhJiL1a1pCA_I1SU7Q,33.508068,-112.037552,35,5.0,AZ,,,,,


In [ ]:
#calculate distance to college campuses and take the five closest ones

import geopy.distance as geo
import numpy as np
dfRestSample = dfRest.sample(n=20000).reset_index(drop= True)
for rest in range(dfRestSample.shape[0]):
    coord1 = (dfRestSample.iloc[rest].latitude, dfRestSample.iloc[rest].longitude)
    closest_dist = np.zeros(7357)
    for i in range(dfCollege.shape[0]):
        coord2 = (dfCollege.iloc[i].latitude, dfCollege.iloc[i].longitude)
        dist = geo.vincenty(coord1, coord2).km
        closest_dist[i] = dist
    closest_dist = np.sort(closest_dist)
    dfRestSample.loc[rest:,('min_dist_1','min_dist_2','min_dist_3','min_dist_4','min_dist_5')] = [closest_dist[0],closest_dist[1],closest_dist[2],closest_dist[3],closest_dist[4]]
dfRestSample.to_csv("rest_sample_state.csv")